In [1]:
%load_ext autoreload
%autoreload 2
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
from qlora import *

import transformers


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/sonia/miniconda3/envs/cleanllama/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda122.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.9
CUDA SETUP: Detected CUDA version 122
CUDA SETUP: Loading binary /home/sonia/miniconda3/envs/cleanllama/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda122.so...


/home/sonia/miniconda3/envs/cleanllama/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /home/sonia/miniconda3/envs/cleanllama did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/home/sonia/miniconda3/envs/cleanllama/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('https'), PosixPath('//debuginfod.ubuntu.com ')}
  warn(msg)
/home/sonia/miniconda3/envs/cleanllama/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('vs/workbench/api/node/extensionHostProcess')}
  warn(msg)
/home/sonia/miniconda3/envs/cleanllama/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in you

In [5]:
argdict = {
  'model_name_or_path' : '/zoo/llama2/llama2-7b-hf',
  'data_seed' : 42 ,
  'dataloader_num_workers' : 1 ,
  'group_by_length' : True,
  'remove_unused_columns' : False ,
  'lora_r' : 64 ,
  'lora_alpha' : 16 ,
  'lora_modules' : 'all' ,
  'double_quant' : True,
  'quant_type' : 'nf4' ,
  'bf16' : True,
  'bits' : 4 ,
  'dataset' : './data/vanilla-adult.dat',
  'dataset_format': 'input-output',
  'source_max_len' : 60 ,
  'target_max_len' : 60 ,
  'seed' : 0
}

arglist = [f'--{k}={v}' for k,v in argdict.items()]

hfparser = transformers.HfArgumentParser((
    ModelArguments, DataArguments, TrainingArguments, GenerationArguments
))
model_args, data_args, training_args, generation_args  = hfparser.parse_args_into_dataclasses(args=arglist, return_remaining_strings=True)[:-1]
training_args.generation_config = transformers.GenerationConfig(**vars(generation_args))
args = argparse.Namespace(
    **vars(model_args), **vars(data_args), **vars(training_args)
)

# Train

In [3]:
checkpoint_dir, completed_training = get_last_checkpoint(args.output_dir)
model, tokenizer = get_accelerate_model(args, checkpoint_dir)
model.config.use_cache = False
    
print('loaded model')
set_seed(args.seed)

loading base model /zoo/llama2/llama2-7b-hf...


/home/sonia/miniconda3/envs/cleanllama/lib/python3.10/site-packages/transformers/modeling_utils.py:2193: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 2/2 [00:32<00:00, 16.42s/it]
/home/sonia/miniconda3/envs/cleanllama/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1714: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Adding special tokens.
adding LoRA modules...
loaded model


In [7]:
data_module = make_data_module(tokenizer=tokenizer, args=args)
collator = data_module['data_collator']
datatr = data_module['train_dataset']

In [8]:
trainer = Seq2SeqTrainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    **{k:v for k,v in data_module.items() if k != 'predict_dataset'},
)

In [13]:
tokenizer.batch_decode([[    1,  4013,  2022, 29915, 29879,  5046,   338, 29871, 29946, 29955,
          7916,   338, 19361,   744,   322,  4234,   338,  4451,  5890,  3148,
         29898,  9485,   314,  3148, 18118,  2992,   467, 13151,  3233,   338,
          3834, 29899,  1054,  4424, 26818,   338, 15710,  8401,   322, 17869,
           338,  1090, 29871, 29945, 29900, 29968,  5046,   338, 29871, 29946,
         29955,  7916,   338, 19361,   744,   322,  4234,   338,  4451,  5890,
          3148, 29898,  9485,   314,  3148],
                  [  5046,   338, 29871, 29946, 29955,
          7916,   338, 19361,   744,   322,  4234,   338,  4451,  5890,  3148,
         29898,  9485,   314,  3148, 18118,  2992,   467, 13151,  3233,   338,
          3834, 29899,  1054,  4424, 26818,   338, 15710,  8401,   322, 17869,
           338,  1090, 29871, 29945, 29900, 29968,  5046,   338, 29871, 29946,
         29955,  7916,   338, 19361,   744,   322,  4234,   338,  4451,  5890,
          3148, 29898,  9485,   314,  3148]])

["<s>This person's age is 47 sex is Female and country is Outlying US(Guam USVI etc). Education level is Some-college occupation is Transport moving and income is under 50K age is 47 sex is Female and country is Outlying US(Guam US",
 'age is 47 sex is Female and country is Outlying US(Guam USVI etc). Education level is Some-college occupation is Transport moving and income is under 50K age is 47 sex is Female and country is Outlying US(Guam US']

In [10]:
trainer.train()

{'input_ids': tensor([[    1,  4013,  2022, 29915, 29879,  5046,   338, 29871, 29946, 29955,
          7916,   338, 19361,   744,   322,  4234,   338,  4451,  5890,  3148,
         29898,  9485,   314,  3148, 18118,  2992,   467, 13151,  3233,   338,
          3834, 29899,  1054,  4424, 26818,   338, 15710,  8401,   322, 17869,
           338,  1090, 29871, 29945, 29900, 29968,  5046,   338, 29871, 29946,
         29955,  7916,   338, 19361,   744,   322,  4234,   338,  4451,  5890,
          3148, 29898,  9485,   314,  3148]]), 'attention_mask': tensor([[True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True]]), 

KeyboardInterrupt: 